In [42]:
import torch
import torch.nn as nn
import sys
from torch.utils.data import DataLoader
from datetime import datetime

sys.path.append("C:/Users/yuuta/Documents/fashion")
from model_learning.model_structure.image_encoder import  ImageEncoderV2
from model_learning.model_structure.dataset import TripletDataset

In [43]:
if torch.cuda.is_available():
    device = torch.device("cuda")  # GPUデバイスを取得
else:
    device = torch.device("cpu")  # CPUデバイスを取得

In [44]:
import importlib
import model_learning
importlib.reload(model_learning.model_structure.image_encoder)

<module 'model_learning.model_structure.image_encoder' from 'C:\\Users/yuuta/Documents/fashion\\model_learning\\model_structure\\image_encoder.py'>

In [45]:
dataset = TripletDataset('C:/Users/yuuta/Documents/fashion/model_learning/compatibility/data/triplet_n_pair.csv', limit=5000000)
test_dataset = TripletDataset('C:/Users/yuuta/Documents/fashion/model_learning/compatibility/data/triplet_n_pair_test.csv', limit=100000)

In [46]:
LEARNING_LATE = 1e-5
BATCH_SIZE = 32
EPOCHS = 10

In [47]:
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

# train_dataset, test_dataset = torch.utils.data.random_split(
#     dataset, [train_size, val_size]
# )

train_dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True)

In [48]:
model = ImageEncoderV2().to(device)
# 
model.load_state_dict(torch.load('C:/Users/yuuta/Documents/fashion/model_learning/compatibility/models/triplet-image-2024-01-05.pth'))
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_LATE)

loss_fn = nn.TripletMarginLoss(margin=20.0, p=2, eps=1e-7) # , reduction='sum'

c:\Users\yuuta\anaconda3\envs\fashion\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\yuuta\anaconda3\envs\fashion\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [49]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (anchor, positive, negative) in enumerate(dataloader):        
        optimizer.zero_grad()

        anchor_vecter = model(anchor.to(device))
        positive_vecter = model(positive.to(device))
        negative_vecter = model(negative.to(device))

        loss = loss_fn(anchor_vecter, positive_vecter, negative_vecter)

        # バックプロパゲーション
        loss.backward()
        optimizer.step()

        if batch % 1000 == 0:
            loss, current = loss.item() / BATCH_SIZE, batch * BATCH_SIZE
            print(f"loss: {loss:>15f}  [{current:>5d}/{size:>5d}]")

        if batch % 3000 == 0:
            now = datetime.now()
            formatted_date = now.strftime("%Y-%m-%d")

            torch.save(model.state_dict(), f'C:/Users/yuuta/Documents/fashion/model_learning/compatibility/models/triplet-image-{formatted_date}.pth')


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss = 0
    p_d = 0
    n_d = 0
    with torch.no_grad():
        for (anchor, positive, negative) in dataloader:
            anchor_vecter = model(anchor.to(device))
            positive_vecter = model(positive.to(device))
            negative_vecter = model(negative.to(device))

            positive_d = torch.norm(anchor_vecter - positive_vecter, dim=1, keepdim=True)
            negative_d = torch.norm(anchor_vecter - negative_vecter, dim=1, keepdim=True)
            p_d += torch.sum(positive_d)
            n_d += torch.sum(negative_d)
            loss = loss_fn(anchor_vecter, positive_vecter, negative_vecter)

            test_loss += loss.item()
            
    test_loss /= size
    print(f"Avg loss: {test_loss:>8f} \n average distance: {(p_d - n_d) / size} posi_ave: {p_d / size} nega_ave: {n_d / size}")

In [50]:
print("start")
for t in range(EPOCHS):
    print(f"Epoch {t+1}\-------------------------------")
    test_loop(test_dataloader, model, loss_fn)
    train_loop(train_dataloader, model, loss_fn, optimizer)
print("Done!")

start
Epoch 1\-------------------------------


Avg loss: 0.059015 
 average distance: -88.95626068115234 posi_ave: 134.961669921875 nega_ave: 223.91793823242188
loss:        0.000000  [    0/5000000]
loss:        0.000000  [32000/5000000]
loss:        0.000000  [64000/5000000]
loss:        0.000000  [96000/5000000]
loss:        0.000000  [128000/5000000]
loss:        0.000000  [160000/5000000]
loss:        0.132434  [192000/5000000]
loss:        0.803391  [224000/5000000]
loss:        0.000000  [256000/5000000]
loss:        0.000000  [288000/5000000]
loss:        0.000000  [320000/5000000]
loss:        0.000000  [352000/5000000]
loss:        0.068520  [384000/5000000]
loss:        0.000000  [416000/5000000]
loss:        0.000000  [448000/5000000]
loss:        0.000000  [480000/5000000]
loss:        0.016785  [512000/5000000]
loss:        0.000000  [544000/5000000]
loss:        0.000000  [576000/5000000]
loss:        0.000000  [608000/5000000]
loss:        0.000000  [640000/5000000]
loss:        0.000000  [672000/5000000]
loss:     

In [ ]:
now = datetime.now()
formatted_date = now.strftime("%Y-%m-%d")

torch.save(model.state_dict(), f'C:/Users/yuuta/Documents/fashion/model_learning/compatibility/models/triplet-image-{formatted_date}.pth')